<a href="https://colab.research.google.com/github/vhmartinsp/sistema_recomendacao/blob/main/sistema_recomendacao_imagens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Configuração do Google Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Colab ou Databricks: suba o arquivo 'kaggle.json'
from google.colab import files
files.upload()  # Isso irá abrir um pop-up para você fazer o upload do arquivo kaggle.json


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"victorhugo08","key":"d08eabeabc30d1c6a1213aa377f66675"}'}

In [3]:
!pip install kaggle


In [4]:
import os

# Criar o diretório .kaggle se não existir
os.makedirs('/root/.kaggle', exist_ok=True)

# Mover o arquivo kaggle.json para o diretório correto
os.rename("/content/kaggle.json", "/root/.kaggle/kaggle.json")

# Garantir as permissões adequadas
os.chmod("/root/.kaggle/kaggle.json", 600)


In [5]:
!kaggle datasets download -d dataclusterlabs/mobile-phone-image-dataset


Dataset URL: https://www.kaggle.com/datasets/dataclusterlabs/mobile-phone-image-dataset
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)
 95% 269M/283M [00:02<00:00, 97.4MB/s]
100% 283M/283M [00:02<00:00, 109MB/s] 


In [6]:
import zipfile

# Descompactar o arquivo ZIP
with zipfile.ZipFile('mobile-phone-image-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('mobile-phone-image-dataset')


In [7]:
import os

# Caminho para o diretório onde as imagens estão localizadas
image_dir = '/content/mobile-phone-image-dataset/Mobile_image/Mobile_image'

# Contar o número de arquivos no diretório
image_count = len([f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))])

image_count


100

In [8]:
!kaggle datasets download -d sugxm00/laptop-dataset


Dataset URL: https://www.kaggle.com/datasets/sugxm00/laptop-dataset
License(s): MIT
 98% 160M/163M [00:02<00:00, 96.0MB/s]
100% 163M/163M [00:02<00:00, 70.5MB/s]


In [9]:
import zipfile
import os

# Caminho onde o arquivo ZIP será salvo
zip_file = '/content/laptop-dataset.zip'

# Caminho de destino para descompactar
destination_folder = '/content/laptop-dataset'

# Descompactar o arquivo ZIP
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)

# Listar o conteúdo da pasta descompactada
os.listdir(destination_folder)


['data']

In [10]:
!kaggle datasets download -d ahedjneed/fancy-watche-images

Dataset URL: https://www.kaggle.com/datasets/ahedjneed/fancy-watche-images
License(s): unknown
 84% 53.0M/63.4M [00:00<00:00, 85.4MB/s]
100% 63.4M/63.4M [00:00<00:00, 95.6MB/s]


In [11]:
import zipfile

# Descompactar o arquivo ZIP
with zipfile.ZipFile('/content/fancy-watche-images.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/fancy-watche-images')


In [12]:
!kaggle datasets download -d dataclusterlabs/suitcaseluggage-dataset

Dataset URL: https://www.kaggle.com/datasets/dataclusterlabs/suitcaseluggage-dataset
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)
 97% 389M/400M [00:04<00:00, 122MB/s]
100% 400M/400M [00:04<00:00, 96.8MB/s]


In [13]:
import zipfile

# Descompactar o arquivo ZIP
with zipfile.ZipFile('/content/suitcaseluggage-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/suitcaseluggage-dataset')


Balancear o número de imagens de cada classe

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import os

# Caminhos das imagens
path_celular = '/content/mobile-phone-image-dataset/Mobile_image/Mobile_image'
path_notebook = '/content/laptop-dataset/data/images'
path_relogio = '/content/fancy-watche-images/content/images'
path_malas = '/content/suitcaseluggage-dataset/suitcase/suitcase'

# Definindo o número desejado de imagens por classe
num_images_target = 3000

# Função para aplicar Data Augmentation
def augment_images(input_folder, output_folder, target_num_images):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Inicializando o gerador de aumento de imagem
    datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Obtendo todas as imagens da pasta de entrada
    images = [f for f in os.listdir(input_folder) if f.endswith('.jpg') or f.endswith('.png')]

    # Gerar as imagens aumentadas
    for image_name in images:
        img_path = os.path.join(input_folder, image_name)
        img = load_img(img_path)  # Carregar a imagem
        x = img_to_array(img)  # Convertê-la em array
        x = x.reshape((1,) + x.shape)  # Redimensionar para formato necessário

        # Gerar imagens aumentadas
        i = 0
        for batch in datagen.flow(x, batch_size=1, save_to_dir=output_folder, save_prefix='aug_', save_format='jpeg'):
            i += 1
            if i >= target_num_images:  # Limitar o número de imagens geradas
                break

# Aplicar o aumento nas classes com menos imagens
augment_images(path_celular, '/content/augmented_mobile_images', 2900)  # Para celular
augment_images(path_notebook, '/content/augmented_laptop_images', 2000)  # Para notebook
augment_images(path_relogio, '/content/augmented_watch_images', 2800)  # Para relógio


In [ ]:
# Verificar o número de imagens em cada pasta
print(f"Celulares: {len(os.listdir('/content/augmented_mobile_images'))}")
print(f"Notebooks: {len(os.listdir('/content/augmented_laptop_images'))}")
print(f"Relógios: {len(os.listdir('/content/augmented_watch_images'))}")
print(f"Malas: {len(os.listdir(path_malas))}")  # Não aplica aumento para malas


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import pickle

# Carregar ResNet50 sem a camada de saída
model = ResNet50(weights="imagenet", include_top=False, pooling="avg", input_shape=(224, 224, 3))

def extract_features(image_path, model):
    """
    Extrai características (features) de uma imagem usando ResNet50.
    """
    img = load_img(image_path, target_size=(224, 224))  # Redimensiona para 224x224
    img_array = img_to_array(img)  # Converte para array NumPy
    img_array = np.expand_dims(img_array, axis=0)  # Adiciona dimensão extra (batch)
    img_array = preprocess_input(img_array)  # Normaliza a imagem para ResNet50
    features = model.predict(img_array)  # Extrai as características
    return features.flatten()  # Retorna um vetor unidimensional

# Criar dicionário para armazenar as características extraídas
feature_db = {}

for category, path in image_dirs.items():
    feature_db[category] = {}
    for image_name in os.listdir(path):
        if image_name.endswith(('.jpg', '.png')):
            image_path = os.path.join(path, image_name)
            feature_db[category][image_name] = extract_features(image_path, model)

# Salvar as características extraídas
with open("features_db.pkl", "wb") as f:
    pickle.dump(feature_db, f)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Carregar o banco de dados de características
with open("features_db.pkl", "rb") as f:
    feature_db = pickle.load(f)

def recommend_similar_images(image_path, model, feature_db, top_n=5):
    """
    Retorna as imagens mais similares à imagem de entrada.
    """
    query_features = extract_features(image_path, model)
    similarities = []

    for category, images in feature_db.items():
        for image_name, features in images.items():
            similarity = cosine_similarity([query_features], [features])[0][0]
            similarities.append((category, image_name, similarity))

    # Ordenar pelas imagens mais similares
    similarities = sorted(similarities, key=lambda x: x[2], reverse=True)

    return similarities[:top_n]  # Retorna top_n imagens similares

# # Teste de recomendação
# test_image = "/content/mobile-phone/Mobile_image/Mobile_image/sample.jpg"
# recommendations = recommend_similar_images(test_image, model, feature_db)

# print("Imagens Recomendadas:")
# for category, image_name, similarity in recommendations:
#     print(f"Categoria: {category}, Imagem: {image_name}, Similaridade: {similarity}")

Testando localmente

In [ ]:
# import matplotlib.pyplot as plt
# import cv2

# def show_recommendations(image_path, recommendations):
#     """
#     Exibe a imagem de entrada e as imagens recomendadas.
#     """
#     fig, axes = plt.subplots(1, len(recommendations) + 1, figsize=(15, 5))

#     # Mostrar a imagem de entrada
#     img = cv2.imread(image_path)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     axes[0].imshow(img)
#     axes[0].set_title("Imagem de Entrada")
#     axes[0].axis("off")

#     # Mostrar imagens recomendadas
#     for i, (category, image_name, _) in enumerate(recommendations):
#         recommended_path = os.path.join(image_dirs[category], image_name)
#         img = cv2.imread(recommended_path)
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#         axes[i + 1].imshow(img)
#         axes[i + 1].set_title(f"Recomendado {i+1}")
#         axes[i + 1].axis("off")

#     plt.show()

# # Testar exibição das recomendações
# show_recommendations(test_image, recommendations)

Testando com a Flask API

In [ ]:
!pip install flask flask-cors tensorflow scikit-learn opencv-python matplotlib


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import os
import pickle
import numpy as np
import tempfile

# Inicializar Flask
app = Flask(_name_)
CORS(app)  # Habilita CORS para permitir acesso de outros domínios

In [ ]:
# Carregar banco de características
with open("features_db.pkl", "rb") as f:
    feature_db = pickle.load(f)

# Diretórios das imagens por categoria
image_dirs = {
    "Celular": "static/celular",
    "Notebook": "static/notebook",
    "Relógio": "static/relogio",
    "Malas": "static/malas"
}

In [ ]:
@app.route("/recommend", methods=["POST"])
def recommend():
    """Recebe uma imagem e retorna recomendações de imagens similares."""
    if "image" not in request.files:
        return jsonify({"error": "Nenhuma imagem enviada"}), 400

    file = request.files["image"]
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".jpg")
    file.save(temp_file.name)

    recommendations = recommend_similar_images(temp_file.name)
    os.remove(temp_file.name)  # Remove o arquivo temporário

    response = []
    for category, image_name, similarity in recommendations:
        image_url = f"/static/{category}/{image_name}"  # Caminho da imagem recomendada
        response.append({"category": category, "image": image_url, "similarity": similarity})

    return jsonify(response)

if _name_ == "_main_":
    app.run(debug=True, host="0.0.0.0", port=5000)

In [ ]:
!python app.py


Criando uma interface WEB com Streamlit

In [ ]:
!pip install streamlit

In [ ]:
import streamlit as st
import requests
import tempfile
import os
from PIL import Image

# URL da API Flask
API_URL = "http://127.0.0.1:5000/recommend"

st.title("🔍 Sistema de Recomendação de Imagens")

st.write("Carregue uma imagem e veja as recomendações mais similares!")

# Upload da imagem
uploaded_file = st.file_uploader("Escolha uma imagem...", type=["jpg", "png", "jpeg"])

if uploaded_file is not None:
    # Criar arquivo temporário
    with tempfile.NamedTemporaryFile(delete=False, suffix=".jpg") as temp_file:
        temp_file.write(uploaded_file.read())
        temp_image_path = temp_file.name

    # Exibir imagem carregada
    st.image(temp_image_path, caption="Imagem de Entrada", use_column_width=True)

    # Enviar imagem para a API Flask
    with open(temp_image_path, "rb") as img_file:
        response = requests.post(API_URL, files={"image": img_file})

    if response.status_code == 200:
        recommendations = response.json()

        st.subheader("Imagens Recomendadas:")
        cols = st.columns(len(recommendations))  # Criar colunas para exibir imagens lado a lado

        for i, rec in enumerate(recommendations):
            image_url = rec["image"]
            similarity = rec["similarity"]
            cols[i].image(image_url, caption=f"Sim: {similarity:.2f}", use_column_width=True)

    os.remove(temp_image_path)  # Remover imagem temporária
!streamlit run streamlit_app.py